In [ ]:
# Convert train.pkl into 1.pkl, 2.pkl, etc.

In [ ]:
# Use model.fit_generator for LSTM firstly, then check using Transformers

In [18]:
import pandas as pd
import numpy as np
import pickle
import os

import tensorflow as tf
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pickle
from os import path
from keras.callbacks import ModelCheckpoint

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
RANDOM_SEED=2245
from numpy.random import seed
seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

from sklearn.metrics import recall_score

In [19]:
def load_pickle(file_name):
    with open(file_name, mode='rb') as fin:
        return pickle.load(fin)

In [20]:
def save_pickle(obj, file_name):
    with open(file_name, mode='wb') as fout:
        pickle.dump(obj, fout)

In [21]:
BASE_FOLDER = os.path.join("/content/drive/MyDrive/Audio classification/mfccs/For-Generator", "train_16_2048_512")
LABELS_FILE = "/content/drive/MyDrive/Audio classification/labels/train_16_2048_512.pkl"

WEIGHTS_FILE = os.path.join("/content/drive/MyDrive/Audio classification/Notebooks-Mahim/WEIGHTS_TESTING")

In [22]:
get_name_pickle = lambda cnt: os.path.join(BASE_FOLDER, "{}.pkl").format(str(cnt))

print(get_name_pickle(1))

/content/drive/MyDrive/Audio classification/mfccs/For-Generator/train_16_2048_512/1.pkl


In [29]:
files = os.listdir(BASE_FOLDER)
print(files[0])
X_0 = load_pickle(os.path.join(BASE_FOLDER, files[0]))
print(X_0.shape)
print(f"len files = {len(files)}")

34561.pkl
(1292, 16)
len files = 35560


In [91]:
print("DUMMY MAKING Y_TRAIN")
Y_train = [np.random.randint(low=0, high=5) for _ in range(len(files))] # high is exclusive
Y_train = tf.keras.utils.to_categorical(Y_train)
print(Y_train.shape, Y_train[0])

DUMMY MAKING Y_TRAIN
(35560, 5) [1. 0. 0. 0. 0.]


## LSTM Model

In [39]:
# input_shape = (X_train.shape[1], X_train.shape[2]) # X_train: batch_size, x[0], x[1]
input_shape = (X_0.shape[0], X_0.shape[1])

# build network topology
model = keras.Sequential()

# 2 LSTM layers
model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
model.add(keras.layers.LSTM(32))
# dense layer
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.3))

# output layer
model.add(keras.layers.Dense(5, activation='softmax'))

## show summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1292, 64)          20736     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total params: 34,373
Trainable params: 34,373
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['acc']) #,tf.keras.metrics.Recall()]
checkpoint = ModelCheckpoint(WEIGHTS_FILE, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [85]:
# Y_train_2 = load_pickle(LABELS_FILE)
# print(Y_train_2.shape)
# print(Y_train_2[0])

(57207, 5)
[0. 1. 0. 0. 0.]


In [93]:
label_32_0 = Y_train[0:32]
label_32_0.shape

(32, 5)

In [94]:
# history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=50, callbacks = callbacks_list, verbose=1) #, class_weight = dict_class_weight)

In [95]:
TOTAL_NUM_FILES = len(files)
print(f"TOTAL_NUM_FILES = {TOTAL_NUM_FILES}")

TOTAL_NUM_FILES = 35560


In [96]:
## https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/#download-the-code
# global count
# count = 1
def my_generator(base_folder, batch_size): # , count=1
    count = 1
    
    while True:
        if count+batch_size < TOTAL_NUM_FILES:
            file_names = [get_name_pickle(cnt) for cnt in range(count, count+batch_size)]
            # print(f"Getting from 1st cond, file_names = {file_names[-1]}")
            objects = [load_pickle(file_name) for file_name in file_names]
            labels = Y_train[count-1: count-1+batch_size]
            yield (np.asarray(objects), np.asarray(labels))
        else:
            count = 1
            file_names = [get_name_pickle(cnt) for cnt in range(count, count+batch_size)]
            # print(f">> Getting from 2nd cond, file_names = {file_names[-1]}")
            objects = [load_pickle(file_name) for file_name in file_names]
            labels = Y_train[count-1: count-1+batch_size]
            yield (np.asarray(objects), np.asarray(labels))

        count += batch_size


########################################################################################
# x, y = my_generator(base_folder=BASE_FOLDER, batch_size=32, count=33)

In [97]:
gen = my_generator(base_folder=BASE_FOLDER, batch_size=2) # , count=33
x, y = gen.__next__()

In [98]:
x.shape, y.shape

((2, 1292, 16), (2, 5))

In [100]:
y

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

## Fit gen model

In [101]:
X_val = load_pickle(file_name = "/content/drive/MyDrive/Audio classification/mfccs/val_16_2048_512.pkl")
Y_val = load_pickle(file_name = "/content/drive/MyDrive/Audio classification/labels/val_16_2048_512.pkl")

In [102]:
print(f"NUM_TRAIN_FILES = {NUM_TRAIN_FILES}")
# print(f"X_val.shape = {X_val.shape}, Y_val.shape = {Y_val.shape}")
Y_val = tf.keras.utils.to_categorical(Y_val)
print(X_val[0].shape, Y_val[0])

NUM_TRAIN_FILES = 35560
(1292, 16) [1. 0. 0. 0. 0.]


In [ ]:
BS = 32 # batch_size = BS = 32
NUM_EPOCHS = 2


history_gen = model.fit(
    x = my_generator(base_folder=BASE_FOLDER, batch_size=BS),
    steps_per_epoch = NUM_TRAIN_FILES // BS,
    validation_data = (X_val, Y_val),
    validation_steps = NUM_TRAIN_FILES // BS,
    epochs = NUM_EPOCHS
)


# H = model.fit(
# 	x=trainGen,
# 	steps_per_epoch=NUM_TRAIN_FILES // BS,
# 	validation_data=testGen,
# 	validation_steps=NUM_TEST_IMAGES // BS,
# 	epochs=NUM_EPOCHS)

Epoch 1/2
   2/1111 [..............................] - ETA: 25:32 - loss: 1.5945 - acc: 0.1875